In [100]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv1D
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [101]:
import pandas as pd
data = pd.read_csv('Uber_Dataset1.csv')
data.head()

,Unnamed: 0,tweet,Subjectivity,Polarity,Analysis,label
0,0,Some very good simulations of bus v car can b...,0.780,0.910,Positive,1
1,1,the all time greatest price and rates ever...,0.875,0.900,Positive,1
2,2,Your services at Bengaluru Airport is so scre...,0.600,0.700,Positive,1
3,3,Lots of good analysis on Uber IPO in the shar...,0.625,0.675,Positive,1
4,4,Seeing this I didnt book any ride from you w...,0.700,0.600,Positive,1


In [102]:
data.drop('Subjectivity',axis=1,inplace=True)
data.drop('Polarity',axis=1,inplace=True)
data.drop('Analysis',axis=1,inplace=True)

In [103]:
data.reset_index(drop=True,inplace=True)

In [104]:
data.head()

,Unnamed: 0,tweet,label
0,0,Some very good simulations of bus v car can b...,1
1,1,the all time greatest price and rates ever...,1
2,2,Your services at Bengaluru Airport is so scre...,1
3,3,Lots of good analysis on Uber IPO in the shar...,1
4,4,Seeing this I didnt book any ride from you w...,1


In [105]:
import string
import re
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct
data['Text_Clean'] = data['tweet'].apply(lambda x: remove_punct(x))

In [106]:
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [107]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [108]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words

In [109]:
pos = []
neg = []
for l in data.label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
data['Pos']= pos
data['Neg']= neg

data = data[['Text_Final', 'tokens', 'label', 'Pos', 'Neg']]
data.head()

,Text_Final,tokens,label,Pos,Neg
0,good simulations bus v car done uber pool even...,"[good, simulations, bus, v, car, done, uber, p...",1,1,0
1,time greatest price rates ever paid company gr...,"[time, greatest, price, rates, ever, paid, com...",1,1,0
2,services bengaluru airport screwed cab drivers...,"[services, bengaluru, airport, screwed, cab, d...",1,1,0
3,lots good analysis uber ipo share price determ...,"[lots, good, analysis, uber, ipo, share, price...",1,1,0
4,seeing didnt book ride went served much better...,"[seeing, didnt, book, ride, went, served, much...",1,1,0


In [110]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.3, 
                                         random_state=78)

In [111]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

2207 words total, with a vocabulary size of 1117
Max sentence length is 36


In [112]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test['tokens']]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

880 words total, with a vocabulary size of 557
Max sentence length is 33


In [113]:
word2vec_path = 'C:/Users/hp/Sentiment-Analysis-RNN-LSTM/GoogleNews-vectors-negative300.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [114]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [115]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train['Text_Final'].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train['Text_Final'].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)

Found 1117 unique tokens.


In [116]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(1118, 300)


In [117]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [2,3,4,5,6]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=10, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs, axis=1)
    x = Dropout(0.1)(l_merge)  
    x = Dense(75, activation='relu')(x)
    #x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [118]:
label_names = ['Pos', 'Neg']

In [119]:
model = ConvNet(train_embedding_weights, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 50, 300)      335400      input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 49, 10)       6010        embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 48, 10)       9010        embedding_5[0][0]                
_______________________________________________________________________________________

In [120]:
y_train = data_train[label_names].values

x_train = train_cnn_data
y_tr = y_train

In [121]:
num_epochs = 5
batch_size = 10
hist = model.fit(x_train, 
                 y_tr, 
                 epochs=num_epochs, 
                 validation_split=0.1, 
                 shuffle=True, 
                 batch_size=batch_size)

Epoch 1/5
11/11 [==============================] - 1s 75ms/step - loss: 0.6731 - acc: 0.6569 - val_loss: 0.6639 - val_acc: 0.5833
Epoch 2/5
11/11 [==============================] - 0s 26ms/step - loss: 0.5864 - acc: 0.6569 - val_loss: 0.6617 - val_acc: 0.5833
Epoch 3/5
11/11 [==============================] - 0s 19ms/step - loss: 0.5564 - acc: 0.7353 - val_loss: 0.6587 - val_acc: 0.5833
Epoch 4/5
11/11 [==============================] - 0s 19ms/step - loss: 0.4682 - acc: 0.7549 - val_loss: 0.7018 - val_acc: 0.5833
Epoch 5/5
11/11 [==============================] - 0s 19ms/step - loss: 0.3985 - acc: 0.8529 - val_loss: 0.7014 - val_acc: 0.5833


In [122]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [123]:
predictions = model.predict(test_cnn_data, 
                            batch_size=3, 
                            verbose=1)
labels = [1, 0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
sum(data_test.label==prediction_labels)/len(prediction_labels)

17/17 [==============================] - 0s 3ms/step


0.6326530612244898